In [1]:
import os
import gc
import torch
import pandas as pd
import numpy as np
import utils.constants as const

from functools import partial

from utils.losses import MosLoss
from utils.xception import Mos_Xception
from utils.data_loader import get_data_loaders
from utils.metrics import accuracy, macro_f1

from fastai.vision.all import *
from fastai.distributed import setup_distrib, num_distrib

/home/bit/miniforge3/envs/pace-up/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.21 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
device = torch.device('cuda:0')

In [3]:
loss = MosLoss(const.NUM_CLASSES)
freeze_bn = False
save_imgs = False
train_losses = []
valid_losses = []
valid_f1s = []
lr_hist = []
metrics = [accuracy, macro_f1]

Focal Loss with gamma =  0


In [4]:
params_df = pd.DataFrame(
    columns=[
        'mom',
        'alpha',
        'eps',
        'wd',
        'f1',
    ],
)

In [5]:
for fold in range(1, 6):
    df = pd.read_csv(f'data/splits/data_fold_{fold}.csv')
    train_dl, val_dl = get_data_loaders(df)
    dls = DataLoaders(train_dl, val_dl)
    result_dict = {}

    for idx in range(20):
        mom = np.random.uniform(0.5, 0.999)
        alpha = np.random.uniform(0.5, 0.999)
        eps = np.random.uniform(1e-6, 0.1)
        wd = np.random.uniform(1e-6, 0.1)

        opt_func = partial(ranger, mom=mom, alpha=alpha, eps=eps)
        net = Mos_Xception(const.NUM_CLASSES)

        learn = Learner(
            dls,
            net,
            wd=wd,
            opt_func=opt_func,
            metrics=metrics,
            loss_func=loss,
        )

        learn.fit_one_cycle(
            5,
            2e-03,
            div=25,
            pct_start=0.3,
        )

        param_dict = {}
        param_dict['mom'] = mom
        param_dict['alpha'] = alpha
        param_dict['eps'] = eps
        param_dict['wd'] = wd
        param_dict['f1'] = learn.metrics[1].value.detach().cpu().numpy()

        result_dict[idx] = param_dict

        del net
        del learn
        gc.collect()
        torch.cuda.empty_cache()

    f1_list = []

    for v in result_dict.values():
        f1_list.append(v['f1'])

    best_idx = np.argmax(f1_list)
    temp_df = pd.DataFrame(
        data=[result_dict[best_idx].values()],
        columns=params_df.columns,
    )
    params_df = pd.concat([params_df, temp_df], ignore_index=True)

epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,4.591326,3.241263,0.330128,0.174324,00:54
1,3.229111,2.680753,0.570513,0.465669,00:49
2,2.498701,1.970669,0.669872,0.531272,00:46
3,2.043194,1.681344,0.717949,0.543925,00:46
4,1.857515,1.658187,0.724359,0.549531,00:44


/home/bit/miniforge3/envs/pace-up/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1716905971190/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,4.347163,3.112251,0.256410,0.113055,00:44
1,2.919575,2.065022,0.660256,0.502275,00:45
2,1.914169,1.326556,0.737179,0.557078,00:44
3,1.444055,1.066550,0.810897,0.613280,00:43
4,1.262608,1.041384,0.810897,0.604664,00:44


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,4.407088,3.152925,0.256410,0.090718,00:43
1,2.998542,2.158340,0.644231,0.533427,00:42
2,2.008740,1.391044,0.782051,0.582106,00:43
3,1.511985,1.125994,0.791667,0.659618,00:42
4,1.329085,1.056377,0.823718,0.646812,00:42


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,4.362395,3.090396,0.288462,0.165783,00:43
1,2.962502,2.053855,0.673077,0.539635,00:43
2,1.989995,1.386625,0.714744,0.543808,00:44
3,1.474903,1.054722,0.823718,0.675039,00:43
4,1.269760,1.086151,0.791667,0.664145,00:42


...

epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,4.660213,3.391407,0.292605,0.129489,00:40
1,3.267452,2.772355,0.549839,0.319594,00:40
2,2.635127,2.151221,0.636656,0.411600,00:40
3,2.217167,1.875315,0.678457,0.466171,00:40
4,2.025317,1.862801,0.665595,0.430651,00:40


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,4.437229,3.185163,0.299035,0.159580,00:40
1,3.127118,2.537104,0.581994,0.383974,00:40
2,2.333743,1.722419,0.745981,0.518314,00:40
3,1.835701,1.514598,0.729904,0.522441,00:40
4,1.619839,1.468094,0.771704,0.540423,00:40


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,4.117237,2.998088,0.360129,0.259698,00:40
1,2.326099,1.177721,0.778135,0.575988,00:40
2,1.325372,0.774228,0.881029,0.669329,00:40
3,0.881567,0.677172,0.868167,0.695424,00:40
4,0.671117,0.480561,0.938907,0.774881,00:40


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,3.985541,2.901630,0.434084,0.279364,00:40
1,2.090354,1.009959,0.752412,0.582169,00:40
2,1.210718,0.858055,0.813505,0.668811,00:40
3,0.838905,0.441495,0.929260,0.764978,00:40
4,0.599854,0.400116,0.945338,0.781246,00:40


epoch,train_loss,valid_loss,accuracy,macro_f1,time
0,4.288332,3.105325,0.299035,0.122195,00:40
1,2.920491,2.074963,0.639871,0.444424,00:40
2,1.974969,1.356169,0.765273,0.553650,00:40
3,1.468474,1.078594,0.826367,0.593209,00:40
4,1.254315,1.055858,0.816720,0.622784,00:40


In [6]:
params_df.to_csv('results/params.csv', index=False)